In [1]:
import plotly
from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd
from utils.load_catch_from_bigquery import load_BQ_and_clean

In [2]:
import geopy.distance
from IPython.display import IFrame

In [3]:
# Add your own plotly user and mapbox access token
plotly.tools.set_credentials_file(username='sollimann', api_key='Mt0jyw4YVAEzihyWlLvL')
plotly.tools.set_config_file(world_readable=True,
                             sharing='public')

mapbox_access_token = 'pk.eyJ1Ijoic29sbGltYW5uIiwiYSI6ImNqeDYyYWVkdDAwYXM0M3QyZ3AwNDJudWUifQ.NaYqXFLNEBG1cVzAdP-GYg'

# ST-DBSCAN

In [4]:
from datetime import datetime
import numpy as np
import ST_DBSCAN.STDBSCAN as STDBSCAN

In [5]:
def filter_catch(catchLimit):
    
    filtered_df = df[(df['Total catch Krill - Mt'] > catchLimit)]
    
    
    return filtered_df

In [6]:
def plot_clusters(df, output_name):
    import matplotlib.pyplot as plt

    labels = df['cluster'].values
    X = df[['Longitude', 'Latitude']].values

    # Black removed and is used for noise instead.
    unique_labels = set(labels)
    colors = [plt.cm.Spectral(each)
              for each in np.linspace(0, 1, len(unique_labels))]
    for k, col in zip(unique_labels, colors):
        if k == -1:
            # Black used for noise.
            col = [0, 0, 0, 1]

        class_member_mask = (labels == k)

        xy = X[class_member_mask]
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                 markeredgecolor='k', markersize=6)

    plt.title('ST-DSCAN: #n of clusters {}'.format(len(unique_labels)))
    plt.show()

# Cluster scatter

In [7]:
def get_consistent_color_scaling(df):
    """
    INPUTS:
        
    
    OUTPUTS:
        
    """
    
    # total number of clusters (noise not included)
    nr_of_clusters = df['cluster'].max()
    
    # all available colors https://www.color-hex.com/
    colors = ["#b8b8b8", "#2791d3", "#27c96d", "#1b5a72", "#eb5e55", "#780303", "#f15e75","#53c4da", "#3b4249",
           "#f7c56e","#ff966c","#ffed43","#177e89","#780303","#51caf9", "#f7c56e", "#f5d4df", "#bee9e8", "#391b55",
              "#c8e4d6",]
    
    # add color for noise manually
    scl = [[0.0,"#b8b8b8"]]
    
    # add additional colors for clusters
    for i in range(1,nr_of_clusters+1):
        index = round(i * (1.0 / nr_of_clusters),3)
        color = [index,colors[i]]
        scl.append(color)

    return scl, colors[0:(nr_of_clusters+1)]

In [8]:
def visualize_data(df,output_name, scl):
    """
    INPUTS:
        
    
    OUTPUTS:
        
    """
    
    # convert a column consisting of datetime64 objects to a string
    df['DateStr'] = df['Date'].dt.strftime('%b') + ' , cluster: ' + df['cluster'].astype(str)

    # Adding dateframes
    df['text'] = df['DateStr'] + ' , ' + df['Vessel'] + ', ' + 'Catch: ' + df['Total catch Krill - Mt'].astype(str)
           
    # data
    data = [
        go.Scattermapbox(
            lon = df['Longitude'],
            lat = df['Latitude'],
            text = df['text'],
            mode='markers',
            marker=go.scattermapbox.Marker(
                size = 8, 
                opacity = 0.8,
                reversescale = False,
                autocolorscale = False,
                colorscale = scl,
                cmin = 0,#df['cluster'].min(),
                color = df['cluster'],
                cmax = df['cluster'].max(),
            ),
        )
    ]

    # Layout
    layout = go.Layout(
        title='catch sites',
        autosize=True,
        hovermode='closest',
        showlegend=True,
        mapbox=go.layout.Mapbox(
            accesstoken=mapbox_access_token,
            bearing=0,
            center=go.layout.mapbox.Center(
                lat=-54,
                lon=-27
            ),
            pitch=0,
            zoom=2,
        ),
    )

    # Return figure
    fig = go.Figure(data=data,layout=layout)
    return fig

# Get cluster data

In [9]:
def total_catch_for_cluster(df_1):

    """
    INPUTS:
        
    
    OUTPUTS:
        
    """

    import calendar
    df_2 = df_1.groupby('month').sum()
    df_2.reset_index(inplace=True)
    df_2['month'] = df_2['month'].apply(lambda x: calendar.month_abbr[x])
    df_2[['month','Total catch Krill - Mt']]
    
    return df_2

In [10]:
def get_average_monthly_catch_from_cluster(df_cluster):
    """
    INPUTS:
        df_clust_id = df[df['cluster' == id]]         a dataframe for a particular cluster
    
    OUTPUTS:
        [catch_month.mean(), ....]   A list of average catch for each month in cluster
    """
    months = []
    df_months = df_cluster['month'].drop_duplicates()
    df_months.index = pd.RangeIndex(len(df_months.index))
    for m in range(0,len(df_months)):
        mean = df_cluster[df_cluster['month'] == df_months[m]].loc[:,"Total catch Krill - Mt"].mean()
        mean = int(mean) #round of
        months.append(str(mean)+' mT')
    return months

In [11]:
def cluster_hist(df):
    """
    INPUTS:
        
    
    OUTPUTS:
        
    """
    import plotly.figure_factory as ff
    hist_data = []
    avg_catch_all_clusters = []
    df_clust = df[df['cluster']==-1]
    df_clust_sorted = df_clust.sort_values(by='month')
    data = df_clust_sorted['Date'].dt.strftime('%b')
    hist_data.append(data)
    
    # add average catch for cluster noise as well
    catch = get_average_monthly_catch_from_cluster(df_clust_sorted)
    avg_catch_all_clusters.append(catch)    
    
    # get data from each cluster
    for c in range(1,max(df_clustered['cluster'])+1):
        df_clust = df[df['cluster']==c]
        df_clust_sorted = df_clust.sort_values(by='month')     
        data = df_clust_sorted['Date'].dt.strftime('%b')
        hist_data.append(data)
        
        # calculate mean catch per month for this cluster
        catch = get_average_monthly_catch_from_cluster(df_clust_sorted)
        avg_catch_all_clusters.append(catch)

    # get cluster label
    group_labels = ['noise']
    for c in range(1,max(df_clustered['cluster'])+1):
        group_labels.append('Cluster ' + str(c))

    return avg_catch_all_clusters,hist_data, group_labels

In [12]:
"""
df = df_clustered
df_clust = df[df['cluster']==c]
df_clust_sorted = df_clust.sort_values(by='month')     
df_clust_sorted
#data = df_clust_sorted['Date'].dt.strftime('%b')
"""

"\ndf = df_clustered\ndf_clust = df[df['cluster']==c]\ndf_clust_sorted = df_clust.sort_values(by='month')     \ndf_clust_sorted\n#data = df_clust_sorted['Date'].dt.strftime('%b')\n"

In [13]:
"""
#def average_catch_pr_month(df_clustered):
df_clust = df_clustered[df_clustered['cluster']==-1]
df_clust_sorted = df_clust.sort_values(by='month') 
#df_clust_sorted
months = []
df_months = df_clust_sorted['month'].drop_duplicates()
df_months.index = pd.RangeIndex(len(df_months.index))

for m in range(0,len(df_months)):
    mean = df_clust_sorted[df_clust_sorted['month'] == df_months[m]].loc[:,"Total catch Krill - Mt"].mean()
    mean = int(mean) #round of
    months.append(str(mean)+' mT')
months
#catch = get_average_monthly_catch_from_cluster(df_clust_sorted)
#catch

#months = get_available_months_from_cluster(df_cluster)
#months
"""

'\n#def average_catch_pr_month(df_clustered):\ndf_clust = df_clustered[df_clustered[\'cluster\']==-1]\ndf_clust_sorted = df_clust.sort_values(by=\'month\') \n#df_clust_sorted\nmonths = []\ndf_months = df_clust_sorted[\'month\'].drop_duplicates()\ndf_months.index = pd.RangeIndex(len(df_months.index))\n\nfor m in range(0,len(df_months)):\n    mean = df_clust_sorted[df_clust_sorted[\'month\'] == df_months[m]].loc[:,"Total catch Krill - Mt"].mean()\n    mean = int(mean) #round of\n    months.append(str(mean)+\' mT\')\nmonths\n#catch = get_average_monthly_catch_from_cluster(df_clust_sorted)\n#catch\n\n#months = get_available_months_from_cluster(df_cluster)\n#months\n'

# Cluster histograms

In [28]:
# number of rows and cols needed
def subplot_rows_and_cols(size):
    """
    INPUTS:
       
    
    OUTPUTS:
        
    """
    import math
    rows = math.ceil(np.sqrt(size))
    cols = rows
    return rows, cols

def create_histograms(avg_catch,hist_data,group_labels, colors):

    """
    INPUTS:
        
    
    OUTPUTS:
        
    """
    
    # init
    data = []
    layout = []
    scl = colors     
    
    # add noise cluster
    noise = go.Histogram(
    x = hist_data[0],
    marker=dict(
    color=scl[0], 
    ),
    opacity = 0.75,
    name = 'noise',
    text = avg_catch[0]
    )
    
    # add noise to data
    data.append(noise)
    
    # add all valid clusters
    for i in range(1,len(hist_data)):
        cluster = go.Histogram(
            x = hist_data[i],
            marker=dict(
            color=scl[i],
            ),
            opacity = 0.75,
            name = group_labels[i],
            text = avg_catch[i]
        )
        data.append(cluster)
    
    # how many rows and cols do we need?
    rows, cols = subplot_rows_and_cols(len(hist_data))
    
    # add clusters to figure
    fig = tools.make_subplots(rows=rows, cols=cols)
    
    # subplot positions
    # max 16 subplots
    row = [1, 1, 2, 2, 1, 2, 3, 3, 3, 1, 2, 3, 4, 4, 4, 4]
    col = [1, 2, 1, 2, 3, 3, 3, 2, 1, 4, 4, 4, 4, 3, 2, 1]
    
    for i in range(0, len(hist_data)):
        fig.append_trace(data[i], row[i], col[i])
        

    fig['layout'].update(height=600, width=600, title='Multiple Subplots' +
                                                      ' with Titles')
    
    
    
    return fig

# Main

In [23]:
%%time
if __name__ == '__main__':
    # Get data
    df = load_BQ_and_clean()
    
    # remove bad catch
    df = filter_catch(100)
    
    # create month
    df['month'] = df['Date'].dt.month
    
    # re-index
    df.index = pd.RangeIndex(len(df.index))
    
    
    spatial_threshold = 10 # km
    temporal_threshold = 1 # months
    min_neighbors = 17
    df_clustered = STDBSCAN.ST_DBSCAN(df, spatial_threshold, temporal_threshold, min_neighbors)
    scl, colors = get_consistent_color_scaling(df_clustered)

Establishing connection to BigQuery in GCP.
    SUCCESS: Connection to BigQuery in GCP established.
Extracting catch data from 2010 to October 2018.
Extracting catch data from December 2018 onwards.
Merging and cleaning catch data
DONE after 8.57sec: Data extracted, merged and cleaned.


C:\Users\ua388\Documents\catch\route\ST_DBSCAN\STDBSCAN.py:38: FutureWarning:

set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead

C:\Users\ua388\Documents\catch\route\ST_DBSCAN\STDBSCAN.py:41: FutureWarning:

set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead

C:\Users\ua388\Documents\catch\route\ST_DBSCAN\STDBSCAN.py:63: FutureWarning:

set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead

C:\Users\ua388\Documents\catch\route\ST_DBSCAN\STDBSCAN.py:34: FutureWarning:

set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead



Wall time: 12min 46s


In [16]:
plot_clusters(df_clustered, 'test')

<Figure size 640x480 with 1 Axes>

In [26]:
fig = visualize_data(df_clustered,'clustered_plot', scl)
py.iplot(fig, filename='catch-sites')

C:\Users\ua388\AppData\Local\Continuum\anaconda3\envs\py37\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [29]:
avg_cluster_catch,hist_data, group_labels = cluster_hist(df_clustered)
avg_cluster_catch
fig = create_histograms(avg_cluster_catch,hist_data, group_labels, colors)
py.iplot(fig, filename='make-subplots-multiple-with-titles')

This is the format of your plot grid:
[ (1,1) x1,y1 ]    [ (1,2) x2,y2 ]    [ (1,3) x3,y3 ]    [ (1,4) x4,y4 ]  
[ (2,1) x5,y5 ]    [ (2,2) x6,y6 ]    [ (2,3) x7,y7 ]    [ (2,4) x8,y8 ]  
[ (3,1) x9,y9 ]    [ (3,2) x10,y10 ]  [ (3,3) x11,y11 ]  [ (3,4) x12,y12 ]
[ (4,1) x13,y13 ]  [ (4,2) x14,y14 ]  [ (4,3) x15,y15 ]  [ (4,4) x16,y16 ]



C:\Users\ua388\AppData\Local\Continuum\anaconda3\envs\py37\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



# Export to CSV 

In [30]:
export_csv = df_clustered.to_csv (r'C:\Users\ua388\Documents\export_dataframe.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path